## Introduction

This notebook serves as a tutorial to show how to use the CalcCorrections class to quickly and easily perform simulation-driven kinematic corrections for any particle, to correct for any systematic drift caused by the reconstuction of data, by comparing what was generated to what was reconstructed.

Approach:
   * Plot 2D histograms of the difference(reconstructed - generated) vs. reconstructed for which we want to correct.
   * The 2D distribution is then "sliced" into bins in the x-axis (reconstructed), and projections in y-axis (difference). 
   * Each slice is then fitted to obtain the mean. 
   * These means are then plotted against the central position of the slice in a scatter graph, which is then fitted to produce a relationship which can be used to perform corrections.


This 'package' has been designed to be largely automated, with the ability to interractively make adjustments on the results of the fits in order to refine them.  The use of Jupyter notebook serves as a shortcut to a GUI, where these adjustments can be made with some simple commands (all of these will be demonstrated below).

Almost all of the required code is "under the hood" such that someone with little-to-no Python experience should be able to use this notebook, and with minimal tweaking, perform corrections of their own. The full code can be viewed in SimKinCorr.py and utils.py. It has been developed for readability, and has been extensively commented.

Python does not have a native histogramming tool as powerful as something like ROOT.  Therefore, we use the [Boost_Histogram](https://github.com/scikit-hep/boost-histogram) package for ease of slicing, and projecting the 2D histograms. It is part of the [scikit-HEP project](https://scikit-hep.org/).

   ---


In [ ]:
import pandas as pd
import uproot
import boost_histogram as bh
import SimKinCorr as skc

#interractive plots in JupyterLab
%matplotlib widget 

#interractive plots in Jupyter-Notebook
#%matplotlib widget
# NB: ONLY LEAVE ONE UNCOMMENTED!

## Reading in data
Data used for this example can be downloaded from: /work/clas12/pnaidoo/CalcCorr/tutorial_data.root

In my case, I performed my analysis using [clas12root](https://github.com/JeffersonLab/clas12root), which means my processed data is in the form of a ROOT file. [Uproot](https://github.com/scikit-hep/uproot) (another packeage from the [scikit-HEP project](https://scikit-hep.org/)) is used to interface between the root file and [pandas-dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) which are commonly used for working with and analysing the data in python.

Pandas is a mature library, with interfaces to many data formats, so this can be adapted to suit your needs if you are confident enough.  Although I have never used it personally, [hipopy](https://github.com/JeffersonLab/hipo_tools#hipopy) is included in hipo-utils, and serves as an interface between the HiPO format and Python.

*__WARNING__: Pandas dataframes hold data RAM. This has the benefit of being very fast, but if you try to open a file larger than the volume of RAM on your system, your system will likely crash.  
As we are using simulated data it is assumed that it is unlikely that this will be an issue on any modern system.  However, there are ways around this, and if demand is sufficient I can implement a work around.*

In [ ]:
#Set destination of data file, and the name of the tree it is stored in.
#-- tree in tutorial_data.root is called "data"
data_file = "/home/pauln/code/neutkincorr/sim_driven/output/ordered.root" 

#data_file = "/PATH/TO/tutorial_data.root" 
data_tree_name = "data"

#Open it with uproot, and convert into a Pandas Dataframe.
data = uproot.open(data_file)[data_tree_name]
df = data.pandas.df() 

# You can quickly view the variable-names/branches you have read in (eg. for easy copy/pasting) by using the command: list(df.columns)
#list(df.columns)

## Set-up
### Perform any calculations required:
In my case, I did not perform the calculations of the difference in reconstructed and generated kinematics.  It's no problem, as this can be done (very quickly) with Pandas.
Each line below appends a new column to our dataframe object, with the result of the calculation. 

In [ ]:
#Calculate difference of reconstructed - generated and append to our dataframe
df['diff_neut_px'] = df.rec_neut_px - df.gen_neut_px;
df['diff_neut_py'] = df.rec_neut_py - df.gen_neut_py;
df['diff_neut_pz'] = df.rec_neut_pz - df.gen_neut_pz;
df['diff_neut_magP'] = df.rec_neut_magP - df.gen_neut_magP;
df['diff_neut_pT'] = df.rec_neut_pT - df.gen_neut_pT;
df['diff_neut_theta'] = df.rec_neut_theta - df.gen_neut_theta;
df['diff_neut_phi'] = df.rec_neut_phi - df.gen_neut_phi;

 match are we talking about here? Are you talking about a regular expression match, or simply a if ... in my_raw_string### Prepare what CalcCorrections needs:
I have tried to make what is required to run the corrections as minimal, and sensible as possible. Other than the data, it requires two things: the cuts you want to perform on the data, and the set up for the variable you wish to correct.

#### Cuts
Cuts should be provided as a list of bracketed statements as shown below.  
Each of these statements return a boolean filter, of equal length as the number of events in our dataframe.  They are combined within the class to apply the cuts to the data when filling the 2D histogram.

Cuts defined below are:
   * $\delta t \le 0.4$ $GeV/c^{2}$
   
   * $\delta\phi_{copl.} \le 5^{\circ}$
   
   * Missing spectator-momentum:  
     $MP_{eD->e'n'\gamma X} \le 0.7$ $ GeV/c^{2}$ 
   
   * $\theta^{cone}_{nX}\le 20^{\circ} $
   
   * Missing total transverse- momentum:  
      $MP^{T}_{en->e'n'\gamma X} \le 0.12$ $ GeV/c^{2}$ 
   
If you performed all of your cutting in your upstream processing, and only have exactly the events which you want to process in your data, you can simply ommit the cuts argument when initialising the class.

#### Set-up Correction
For this, we use a python dictionary - a special kind of container which contains labeled values. 
Below we set up $p_{x}$, and the fields of the dictionary are:
   * `var`: The name of the variable in the dataframe we wish to correct.
   * `v_bins_range`: A tuple which contains (# of bins, low range limit, high range limit) for `var`.
   * `v_label`: A plot label for `var` which will be used in the figure.
   * `delta`: The name of the difference of the target variable we are correcting.
   * `d_bins_range`: A tuple which contains (# of bins, low range limit, high range limit) for `delta`.
   * `d_label`: A plot label for `delta` which will be used in the figure.

Every field in this setup object must be supplied, and everything on the left hand side of the colon should be left as is, as these labels are used in the inner working of the class.

In [ ]:
cuts = [
    (abs(df.rec_dt) <= 0.4),
    (abs(df.rec_cop) <= 5),
    (df.rec_MPspect <= 0.7),
    (df.rec_neutcone <= 20),
    (df.rec_MPT_tot <= 0.12)
]


px = {
    'var': 'rec_neut_px',
    'v_bins_range': (70, -0.5, 0.5),
    'v_label': 'rec $p_{x}$',
    'delta': 'diff_neut_px',
    'd_bins_range': (70, -0.1, 0.1),
    'd_label': '$\delta p_{x}$ (recon - gen)'
}

## Corrections
### Running Corrections 
With our cuts and our setup defined, we are ready! 
Remember to assign the instance to a variable so that we can make adjustments to anything which needs it.

In [ ]:
corr_px = skc.CalcCorrections(data=df, setup=px, cuts=cuts)
# you can also specify how many slices you would like by passing "n_slices = N" as an argument. Default is 10.

### Adjusting slices
Once we have produced our instance of the class, it is time to visually inspect the fits and Chi-Squareds, and see if we need to make any changes.
We have several options to manually tweak any slice which was either fitted poorly, or whose fit failed. 

Individual slices can:
   * Have their initial fit parameters hard coded
   * Have their fit range changed
   * Be rebinned if a particular slice has slow stats
   * If you want to revert any of the above changes, the slice can be 
       restored.
   * If a slice cannot be 'made to cooperate' it can be vetoed, meaning it 
       will not be included in the final fit of the means.  
       _NB: slices can also be 'un-vetoed'._

In every case, the distribution of the means is replotted and refitted, with the result updating in the figure.

See the cells below for an example of each adjustment.

In [ ]:
#Rebin slice 9 by a factor 2.
corr_px.rebin_slice(slc_idx=9, factor=2)

In [ ]:
# Limit fit range of slice 3 to between -0.04 and 0.25.
corr_px.set_slice_fitrange(slc_idx=3, low=-0.04, high=0.025)

In [ ]:
# Restore slice 3 to its original fit
corr_px.restore_slice(3)

In [ ]:
# Restore slice 9 to its original binning
corr_px.restore_slice(9)

In [ ]:
# Hard code the parameters for slice 9
corr_px.set_slice_fitparams(slc_idx=9, amp=10, mean=-0.01, sigma=0.1, const=5)

In [ ]:
# Veto slice 9 from consideration
corr_px.veto_slice(9)

In [ ]:
# Unveto slice 9 so it is included in the fit
corr_px.unveto_slice(9)

In [ ]:
# Change the model used to fit the final result
# Default: linear.  Options: po2, pol3. pol4
corr_px.change_fit_model("pol3")

In [ ]:
# Use an arbitrary model to fit the final result
def model(x, k3, c):
    return k3*x**3 + c

corr_px.user_def_model(model)

### Validating Corrections 
We want to validate our corrections.  We do this by applying them to our simulated data. It is incredibly quick and easy to do this using the power of Panda's dataframes, and the CalcCorrections class! 

The fitmodel used, and the parameter values of that model from our results are stored in the instance of our class, `corr_px`!  Therefore in one line we can append a new column to our dataframe with the result of the correction.

We can then simply reprocess the data as we did before.  If the correction worked, we expect values to be more-centred on, and less spread around zero.
All we need to do is define a set-up dictionary and make a new instance of the class.

In [ ]:
# for every value in rec_neut_px, subtract f(rec_neut_px), using the resulting fit-values.
df['corr_neut_px'] = df.rec_neut_px - corr_px.fit_model(df.rec_neut_px, *corr_px.fit_vals)

# recalculate the difference between reconstructed and generated post-correction.
df['corr_diff_neut_px'] = df.corr_neut_px- df.gen_neut_px;

In [ ]:
# Set-up dictionary
corrected_px = {
    'var': 'corr_neut_px',
    'v_bins_range': (70, -0.5, 0.5),
    'v_label': 'corr. rec $p_{x}$',
    'delta': 'corr_diff_neut_px',
    'd_bins_range': (70, -0.1, 0.1),
    'd_label': 'corr. $\delta p_{x}$ (recon - gen)'
}

In [ ]:
# create new instance of the class
val_px = skc.CalcCorrections(data=df, setup=corrected_px, cuts=cuts)